# Trabajar con datos de series temporales

## Acerca de los datos
En este cuaderno trabajaremos con 5 conjuntos de datos:
- (CSV) Cotización diaria de las acciones de Facebook a lo largo de 2018 (obtenida mediante el método [`stock_analysis` package](https://github.com/stefmolin/stock-analysis)).
- (CSV) Datos bursátiles OHLC de Facebook del 20 de mayo de 2019 al 24 de mayo de 2019 por minuto de [Nasdaq.com](https://old.nasdaq.com/symbol/fb/interactive-chart).
- (CSV) datos de acciones fundidos para Facebook del 20 de mayo de 2019 al 24 de mayo de 2019 por minuto de [Nasdaq.com](https://old.nasdaq.com/symbol/fb/interactive-chart).
- (DB) precios de apertura de las acciones por minuto para Apple del 20 de mayo de 2019 al 24 de mayo de 2019 alterados para tener segundos en el tiempo de [Nasdaq.com](https://old.nasdaq.com/symbol/aapl/interactive-chart).
- (DB) precios de apertura de las acciones de Facebook por minuto del 20 de mayo de 2019 al 24 de mayo de 2019 de [Nasdaq.com](https://old.nasdaq.com/symbol/fb/interactive-chart).

## Setup

In [1]:
import numpy as np
import pandas as pd

fb = pd.read_csv('data/fb_2018.csv', index_col='date', parse_dates=True).assign(
    trading_volume=lambda x: pd.cut(x.volume, bins=3, labels=['low', 'med', 'high'])
)
fb.head()

,open,high,low,close,volume,trading_volume
date,,,,,,
2018-01-02,177.68,181.58,177.5500,181.42,18151903,low
2018-01-03,181.88,184.78,181.3300,184.67,16886563,low
2018-01-04,184.90,186.21,184.0996,184.33,13880896,low
2018-01-05,185.59,186.90,184.9300,186.85,13574535,low
2018-01-08,187.20,188.90,186.3300,188.28,17994726,low


## Selección y filtrado por tiempo
Recuerda, cuando tenemos un índice de tipo `DatetimeIndex`, podemos utilizar el corte por fechas. Podemos proporcionar un rango de fechas. Sólo recuperamos tres días porque la bolsa cierra los fines de semana:

In [2]:
fb['2018-10-11':'2018-10-15']

,open,high,low,close,volume,trading_volume
date,,,,,,
2018-10-11,150.13,154.81,149.1600,153.35,35338901,low
2018-10-12,156.73,156.89,151.2998,153.74,25293492,low
2018-10-15,153.32,155.57,152.5500,153.52,15433521,low


Podemos seleccionar rangos de meses y trimestres:

In [3]:
fb.loc['2018-q1'].equals(fb['2018-01':'2018-03'])

True

El método `first()` nos dará una longitud de tiempo especificada desde el principio de la serie temporal. Aquí, pedimos una semana. El 1 de enero de 2018 era festivo, es decir, el mercado estaba cerrado. También fue un lunes, por lo que la semana aquí es de sólo cuatro días:

In [4]:
fb.first('1W')

C:\Users\feder\AppData\Local\Temp\ipykernel_135348\2655357208.py:1: FutureWarning: first is deprecated and will be removed in a future version. Please create a mask and filter using `.loc` instead
  fb.first('1W')


,open,high,low,close,volume,trading_volume
date,,,,,,
2018-01-02,177.68,181.58,177.5500,181.42,18151903,low
2018-01-03,181.88,184.78,181.3300,184.67,16886563,low
2018-01-04,184.90,186.21,184.0996,184.33,13880896,low
2018-01-05,185.59,186.90,184.9300,186.85,13574535,low


El método `last()` tomará del final:

In [5]:
fb.last('1W')

C:\Users\feder\AppData\Local\Temp\ipykernel_135348\2477606097.py:1: FutureWarning: last is deprecated and will be removed in a future version. Please create a mask and filter using `.loc` instead
  fb.last('1W')


,open,high,low,close,volume,trading_volume
date,,,,,,
2018-12-31,134.45,134.64,129.95,131.09,24625308,low


Supongamos que reindexamos los datos de las acciones de Facebook para incluir todas las fechas de 2018. Tendríamos entradas nulas para el 1 de enero:

In [6]:
fb_reindexed = fb.reindex(pd.date_range('2018-01-01', '2018-12-31', freq='D'))
fb_reindexed.first('1D').isna().squeeze().all()

C:\Users\feder\AppData\Local\Temp\ipykernel_135348\283175235.py:2: FutureWarning: first is deprecated and will be removed in a future version. Please create a mask and filter using `.loc` instead
  fb_reindexed.first('1D').isna().squeeze().all()


True

Podemos utilizar `first_valid_index()` para obtener el índice de la primera entrada no nula de nuestros datos, que es el primer día de apertura del mercado en el primer trimestre de 2018:

In [7]:
fb_reindexed.loc['2018-Q1'].first_valid_index()

Timestamp('2018-01-02 00:00:00')

A la inversa, podemos utilizar `last_valid_index()` para obtener la última entrada de datos no nulos. Para el primer trimestre de 2018, es el 29 de marzo:

In [8]:
fb_reindexed.loc['2018-Q1'].last_valid_index()

Timestamp('2018-03-29 00:00:00')

Podemos utilizar `asof()` para encontrar el último dato no nulo anterior al punto que buscamos. Si pedimos el 31 de marzo, obtendremos los datos del índice que obtuvimos de `fb_reindexed.loc['2018-Q1'].last_valid_index()`, que fue el 29 de marzo. Tenga en cuenta que esto funciona independientemente de si hemos reindexado:

In [9]:
fb_reindexed.asof('2018-03-31')

open                  155.15
high                  161.42
low                   154.14
close                 159.79
volume            59434293.0
trading_volume           low
Name: 2018-03-31 00:00:00, dtype: object

Para los siguientes ejemplos, necesitamos fechas y horas, por lo que leeremos el archivo de datos de acciones por minuto:

In [10]:
stock_data_per_minute = pd.read_csv(
    'data/fb_week_of_may_20_per_minute.csv', index_col='date', parse_dates=True, 
    date_parser=lambda x: pd.to_datetime(x, format='%Y-%m-%d %H-%M')
)

stock_data_per_minute.head()

C:\Users\feder\AppData\Local\Temp\ipykernel_135348\2041232261.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  stock_data_per_minute = pd.read_csv(


,open,high,low,close,volume
date,,,,,
2019-05-20 09:30:00,181.6200,181.6200,181.6200,181.6200,159049.0
2019-05-20 09:31:00,182.6100,182.6100,182.6100,182.6100,468017.0
2019-05-20 09:32:00,182.7458,182.7458,182.7458,182.7458,97258.0
2019-05-20 09:33:00,182.9500,182.9500,182.9500,182.9500,43961.0
2019-05-20 09:34:00,183.0600,183.0600,183.0600,183.0600,79562.0


Podemos utilizar un objeto `Grouper` para enrollar nuestros datos a nivel diario junto con `first` y `last`:

In [11]:
stock_data_per_minute.groupby(pd.Grouper(freq='1D')).agg({
    'open': 'first',
    'high': 'max', 
    'low': 'min', 
    'close': 'last', 
    'volume': 'sum'
})

,open,high,low,close,volume
date,,,,,
2019-05-20,181.62,184.1800,181.6200,182.72,10044838.0
2019-05-21,184.53,185.5800,183.9700,184.82,7198405.0
2019-05-22,184.81,186.5603,184.0120,185.32,8412433.0
2019-05-23,182.50,183.7300,179.7559,180.87,12479171.0
2019-05-24,182.33,183.5227,181.0400,181.06,7686030.0


El método `at_time()` nos permite extraer todas las fechas que coincidan con una hora determinada. En este caso, podemos obtener todas las filas a partir de la hora de apertura de la bolsa (9:30 AM):

In [12]:
stock_data_per_minute.at_time('9:30')

,open,high,low,close,volume
date,,,,,
2019-05-20 09:30:00,181.62,181.62,181.62,181.62,159049.0
2019-05-21 09:30:00,184.53,184.53,184.53,184.53,58171.0
2019-05-22 09:30:00,184.81,184.81,184.81,184.81,41585.0
2019-05-23 09:30:00,182.50,182.50,182.50,182.50,121930.0
2019-05-24 09:30:00,182.33,182.33,182.33,182.33,52681.0


Podemos utilizar `between_time()` para obtener los datos de los dos últimos minutos de la negociación diaria:

In [13]:
stock_data_per_minute.between_time('15:59', '16:00')

,open,high,low,close,volume
date,,,,,
2019-05-20 15:59:00,182.915,182.915,182.915,182.915,134569.0
2019-05-20 16:00:00,182.720,182.720,182.720,182.720,1113672.0
2019-05-21 15:59:00,184.840,184.840,184.840,184.840,61606.0
2019-05-21 16:00:00,184.820,184.820,184.820,184.820,801080.0
2019-05-22 15:59:00,185.290,185.290,185.290,185.290,96099.0
2019-05-22 16:00:00,185.320,185.320,185.320,185.320,1220993.0
2019-05-23 15:59:00,180.720,180.720,180.720,180.720,109648.0
2019-05-23 16:00:00,180.870,180.870,180.870,180.870,1329217.0
2019-05-24 15:59:00,181.070,181.070,181.070,181.070,52994.0


Por término medio, ¿se negocian más acciones en los primeros 30 minutos de negociación o en los últimos 30 minutos? Podemos combinar `between_time()` con `group_by()` y `filter()` de la función [`3-aggregations.ipynb`](./3-aggregations.ipynb) para responder a esta pregunta. En la semana en cuestión, se negocia más por término medio a la hora de apertura que a la hora de cierre:

In [14]:
shares_traded_in_first_30_min = stock_data_per_minute\
    .between_time('9:30', '10:00')\
    .groupby(pd.Grouper(freq='1D'))\
    .filter(lambda x: (x.volume > 0).all())\
    .volume.mean()

shares_traded_in_last_30_min = stock_data_per_minute\
    .between_time('15:30', '16:00')\
    .groupby(pd.Grouper(freq='1D'))\
    .filter(lambda x: (x.volume > 0).all())\
    .volume.mean()

shares_traded_in_first_30_min - shares_traded_in_last_30_min

18592.967741935485

En los casos en que la hora no importa, podemos normalizar las horas a medianoche:

In [15]:
pd.DataFrame(
    dict(before=stock_data_per_minute.index, after=stock_data_per_minute.index.normalize())
).head()

,before,after
0,2019-05-20 09:30:00,2019-05-20
1,2019-05-20 09:31:00,2019-05-20
2,2019-05-20 09:32:00,2019-05-20
3,2019-05-20 09:33:00,2019-05-20
4,2019-05-20 09:34:00,2019-05-20


Tenga en cuenta que también podemos utilizar `normalize()` en un objeto `Series` después de acceder al atributo `dt`:

In [16]:
stock_data_per_minute.index.to_series().dt.normalize().head()

date
2019-05-20 09:30:00   2019-05-20
2019-05-20 09:31:00   2019-05-20
2019-05-20 09:32:00   2019-05-20
2019-05-20 09:33:00   2019-05-20
2019-05-20 09:34:00   2019-05-20
Name: date, dtype: datetime64[ns]

## Desplazamiento para datos retardados
Podemos utilizar `shift()` para crear datos retardados. Por defecto, el desplazamiento será de un periodo. Por ejemplo, podemos utilizar `shift()` para crear una nueva columna que indique el precio de cierre del día anterior. A partir de esta nueva columna, podemos calcular el cambio de precio debido a la negociación fuera de horario (después del cierre de un día hasta la apertura del día siguiente):

In [17]:
fb.assign(
    prior_close=lambda x: x.close.shift(),
    after_hours_change_in_price=lambda x: x.open - x.prior_close,
    abs_change=lambda x: x.after_hours_change_in_price.abs()
).nlargest(5, 'abs_change')

,open,high,low,close,volume,trading_volume,prior_close,after_hours_change_in_price,abs_change
date,,,,,,,,,
2018-07-26,174.89,180.13,173.75,176.26,169803668,high,217.50,-42.61,42.61
2018-04-26,173.22,176.27,170.80,174.16,77556934,med,159.69,13.53,13.53
2018-01-12,178.06,181.48,177.40,179.37,77551299,med,187.77,-9.71,9.71
2018-10-31,155.00,156.40,148.96,151.79,60101251,low,146.22,8.78,8.78
2018-03-19,177.01,177.17,170.06,172.56,88140060,med,185.09,-8.08,8.08


Si el objetivo es sumar/restar tiempo, podemos utilizar objetos `pd.Timedelta` en su lugar:

In [18]:
pd.date_range('2018-01-01', freq='D', periods=5) + pd.Timedelta('9 hours 30 minutes')

DatetimeIndex(['2018-01-01 09:30:00', '2018-01-02 09:30:00',
               '2018-01-03 09:30:00', '2018-01-04 09:30:00',
               '2018-01-05 09:30:00'],
              dtype='datetime64[ns]', freq='D')

## Datos diferenciados
Utilizar el método `diff()` es una forma rápida de calcular la diferencia entre los datos y una versión retardada de los mismos. Por defecto, dará el resultado de `data - data.shift()`:

In [19]:
(
    fb.drop(columns='trading_volume') 
    - fb.drop(columns='trading_volume').shift()
).equals(
    fb.drop(columns='trading_volume').diff()
)

True

Podemos utilizarlo para ver la evolución diaria de las acciones de Facebook:

In [20]:
fb.drop(columns='trading_volume').diff().head()

,open,high,low,close,volume
date,,,,,
2018-01-02,NaN,NaN,NaN,NaN,NaN
2018-01-03,4.20,3.20,3.7800,3.25,-1265340.0
2018-01-04,3.02,1.43,2.7696,-0.34,-3005667.0
2018-01-05,0.69,0.69,0.8304,2.52,-306361.0
2018-01-08,1.61,2.00,1.4000,1.43,4420191.0


Podemos especificar el número de períodos, puede ser cualquier número entero positivo o negativo:

In [21]:
fb.drop(columns='trading_volume').diff(-3).head()

,open,high,low,close,volume
date,,,,,
2018-01-02,-7.91,-5.32,-7.3800,-5.43,4577368.0
2018-01-03,-5.32,-4.12,-5.0000,-3.61,-1108163.0
2018-01-04,-3.80,-2.59,-3.0004,-3.54,1487839.0
2018-01-05,-1.35,-0.99,-0.7000,-0.99,3044641.0
2018-01-08,-1.20,0.50,-1.0500,0.51,8406139.0


## Remuestreo
A veces los datos tienen una granularidad que no es propicia para nuestro análisis. Consideremos el caso en el que tenemos datos por minuto para todo el año 2018. Veamos qué ocurre si intentamos graficar esto, y luego observamos la agregación diaria de estos datos.

In [22]:
from visual-aids.visual_aids.misc_viz import resampling_example
resampling_example()

SyntaxError: invalid syntax (3289253952.py, line 1)

El gráfico de la izquierda tiene tantos datos que no se ve nada. Sin embargo, cuando agregamos los totales diarios, vemos los datos. Podemos modificar la granularidad de los datos con los que trabajamos utilizando el remuestreo. Recordemos nuestros datos bursátiles minuto a minuto:

In [23]:
stock_data_per_minute.head()

,open,high,low,close,volume
date,,,,,
2019-05-20 09:30:00,181.6200,181.6200,181.6200,181.6200,159049.0
2019-05-20 09:31:00,182.6100,182.6100,182.6100,182.6100,468017.0
2019-05-20 09:32:00,182.7458,182.7458,182.7458,182.7458,97258.0
2019-05-20 09:33:00,182.9500,182.9500,182.9500,182.9500,43961.0
2019-05-20 09:34:00,183.0600,183.0600,183.0600,183.0600,79562.0


Podemos remuestrear esto para llegar a una frecuencia diaria:

In [24]:
stock_data_per_minute.resample('1D').agg({
    'open': 'first',
    'high': 'max', 
    'low': 'min', 
    'close': 'last', 
    'volume': 'sum'
})

,open,high,low,close,volume
date,,,,,
2019-05-20,181.62,184.1800,181.6200,182.72,10044838.0
2019-05-21,184.53,185.5800,183.9700,184.82,7198405.0
2019-05-22,184.81,186.5603,184.0120,185.32,8412433.0
2019-05-23,182.50,183.7300,179.7559,180.87,12479171.0
2019-05-24,182.33,183.5227,181.0400,181.06,7686030.0


Podemos reducir la muestra a datos trimestrales:

In [ ]:
fb.resample('Q').mean()

También podemos utilizar `apply()`. Aquí, mostramos el cambio trimestral de principio a fin:

In [26]:
fb.drop(columns='trading_volume').resample('Q').apply(
    lambda x: x.last('1D').values - x.first('1D').values
)

C:\Users\feder\AppData\Local\Temp\ipykernel_135348\816813945.py:2: FutureWarning: last is deprecated and will be removed in a future version. Please create a mask and filter using `.loc` instead
  lambda x: x.last('1D').values - x.first('1D').values
C:\Users\feder\AppData\Local\Temp\ipykernel_135348\816813945.py:2: FutureWarning: first is deprecated and will be removed in a future version. Please create a mask and filter using `.loc` instead
  lambda x: x.last('1D').values - x.first('1D').values


date
2018-03-31    [[-22.53, -20.160000000000025, -23.41000000000...
2018-06-30    [[39.50999999999999, 38.399700000000024, 39.84...
2018-09-30    [[-25.039999999999992, -28.659999999999997, -2...
2018-12-31    [[-28.580000000000013, -31.24000000000001, -31...
Freq: Q-DEC, dtype: object

Considere los siguientes datos de acciones fundidas por minutos. No vemos los datos OHLC directamente:

In [27]:
melted_stock_data = pd.read_csv('data/melted_stock_data.csv', index_col='date', parse_dates=True)
melted_stock_data.head()

,price
date,
2019-05-20 09:30:00,181.6200
2019-05-20 09:31:00,182.6100
2019-05-20 09:32:00,182.7458
2019-05-20 09:33:00,182.9500
2019-05-20 09:34:00,183.0600


Podemos utilizar el método `ohlc()` después del remuestreo para recuperar las columnas OHLC:

In [28]:
melted_stock_data.resample('1D').ohlc()['price']

,open,high,low,close
date,,,,
2019-05-20,181.62,184.1800,181.6200,182.72
2019-05-21,184.53,185.5800,183.9700,184.82
2019-05-22,184.81,186.5603,184.0120,185.32
2019-05-23,182.50,183.7300,179.7559,180.87
2019-05-24,182.33,183.5227,181.0400,181.06


Alternativamente, podemos aumentar la muestra para aumentar la granularidad. Tenga en cuenta que esto introducirá valores `NaN`:

In [29]:
fb.resample('6H').asfreq().head()

,open,high,low,close,volume,trading_volume
date,,,,,,
2018-01-02 00:00:00,177.68,181.58,177.55,181.42,18151903.0,low
2018-01-02 06:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-02 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-02 18:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-03 00:00:00,181.88,184.78,181.33,184.67,16886563.0,low


Hay muchas maneras de manejar estos valores `NaN`. Podemos rellenar con `pad()`:

In [31]:
fb.resample('6H').pad().head()

AttributeError: 'DatetimeIndexResampler' object has no attribute 'pad'

Podemos especificar un valor concreto o un método con `fillna()`:

In [32]:
fb.resample('6H').fillna('nearest').head()

C:\Users\feder\AppData\Local\Temp\ipykernel_135348\3498320007.py:1: FutureWarning: DatetimeIndexResampler.fillna is deprecated and will be removed in a future version. Use obj.ffill(), obj.bfill(), or obj.nearest() instead.
  fb.resample('6H').fillna('nearest').head()


,open,high,low,close,volume,trading_volume
date,,,,,,
2018-01-02 00:00:00,177.68,181.58,177.55,181.42,18151903,low
2018-01-02 06:00:00,177.68,181.58,177.55,181.42,18151903,low
2018-01-02 12:00:00,181.88,184.78,181.33,184.67,16886563,low
2018-01-02 18:00:00,181.88,184.78,181.33,184.67,16886563,low
2018-01-03 00:00:00,181.88,184.78,181.33,184.67,16886563,low


Podemos utilizar `asfreq()` y `assign()` para especificar la acción por columna:

In [33]:
fb.resample('6H').asfreq().assign(
    volume=lambda x: x.volume.fillna(0), # poner 0 cuando el mercado está cerrado
    close=lambda x: x.close.fillna(method='ffill'), # llevar adelante
    # tomar el precio de cierre si estos no están disponibles
    open=lambda x: x.open.combine_first(x.close),
    high=lambda x: x.high.combine_first(x.close),
    low=lambda x: x.low.combine_first(x.close)
).head()

C:\Users\feder\AppData\Local\Temp\ipykernel_135348\2905588721.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  close=lambda x: x.close.fillna(method='ffill'), # llevar adelante


,open,high,low,close,volume,trading_volume
date,,,,,,
2018-01-02 00:00:00,177.68,181.58,177.55,181.42,18151903.0,low
2018-01-02 06:00:00,181.42,181.42,181.42,181.42,0.0,NaN
2018-01-02 12:00:00,181.42,181.42,181.42,181.42,0.0,NaN
2018-01-02 18:00:00,181.42,181.42,181.42,181.42,0.0,NaN
2018-01-03 00:00:00,181.88,184.78,181.33,184.67,16886563.0,low


## Merging
Vimos ejemplos de fusión en el [`1-querying_and_merging.ipynb`](./1-querying_and_merging.ipynb). Sin embargo, todas coincidían en función de las claves. Con las series temporales, es posible que sean tan granulares que nunca tengamos la misma hora para varias entradas. Trabajemos con algunos datos bursátiles en diferentes granularidades:

In [34]:
import sqlite3

with sqlite3.connect('data/stocks.db') as connection:
    fb_prices = pd.read_sql(
        'SELECT * FROM fb_prices', connection, 
        index_col='date', parse_dates=['date']
    )
    aapl_prices = pd.read_sql(
        'SELECT * FROM aapl_prices', connection, 
        index_col='date', parse_dates=['date']
    )

Los precios de Facebook son al minuto:

In [35]:
fb_prices.index.second.unique()

Index([0], dtype='int32', name='date')

Sin embargo, los precios de Apple tienen información para el segundo:

In [36]:
aapl_prices.index.second.unique()

Index([ 0, 52, 36, 34, 55, 35,  7, 12, 59, 17,  5, 20, 26, 23, 54, 49, 19, 53,
       11, 22, 13, 21, 10, 46, 42, 38, 33, 18, 16,  9, 56, 39,  2, 50, 31, 58,
       48, 24, 29,  6, 47, 51, 40,  3, 15, 14, 25,  4, 43,  8, 32, 27, 30, 45,
        1, 44, 57, 41, 37, 28],
      dtype='int32', name='date')

Podemos realizar una fusión *as of* para intentar alinearlas lo mejor posible. Especificamos cómo manejar el desajuste con los parámetros `direction` y `tolerance`. Vamos a rellenar con la "dirección" de "más cercano" y una "tolerancia" de 30 segundos. Esto colocará los datos de Apple con el minuto al que esté más cerca, así que 9:31:52 irá con 9:32 y 9:37:07 irá con 9:37. Como los tiempos están en el índice, pasamos `left_index` y `right_index`, como hicimos con `merge()` anteriormente en este capítulo:

In [37]:
pd.merge_asof(
    fb_prices, aapl_prices, 
    left_index=True, right_index=True, # las fechas están en el índice
    # fusionar con el minuto más cercano
    direction='nearest', tolerance=pd.Timedelta(30, unit='s')
).head()

,FB,AAPL
date,,
2019-05-20 09:30:00,181.6200,183.5200
2019-05-20 09:31:00,182.6100,NaN
2019-05-20 09:32:00,182.7458,182.8710
2019-05-20 09:33:00,182.9500,182.5000
2019-05-20 09:34:00,183.0600,182.1067


Si no queremos perder la información de los segundos con los datos de Apple, podemos utilizar `pd.merge_ordered()` en su lugar, que intercalará los dos. Tenga en cuenta que se trata de una unión externa por defecto (parámetro `how`). La única pega es que tenemos que restablecer el índice para poder unirnos a él:

In [38]:
pd.merge_ordered(
    fb_prices.reset_index(), aapl_prices.reset_index()
).set_index('date').head()

,FB,AAPL
date,,
2019-05-20 09:30:00,181.6200,183.520
2019-05-20 09:31:00,182.6100,NaN
2019-05-20 09:31:52,NaN,182.871
2019-05-20 09:32:00,182.7458,NaN
2019-05-20 09:32:36,NaN,182.500


Podemos pasar un `fill_method` para manejar valores `NaN`:

In [39]:
pd.merge_ordered(
    fb_prices.reset_index(), aapl_prices.reset_index(),
    fill_method='ffill'
).set_index('date').head()

,FB,AAPL
date,,
2019-05-20 09:30:00,181.6200,183.520
2019-05-20 09:31:00,182.6100,183.520
2019-05-20 09:31:52,182.6100,182.871
2019-05-20 09:32:00,182.7458,182.871
2019-05-20 09:32:36,182.7458,182.500


Alternativamente, podemos utilizar `fillna()`.

<hr>

<div style="display: flex; justify-content: space-between; margin-bottom: 10px;">
    <div style="text-align: left;">
        <a href="./3-agregaciones.ipynb">
            <button>&#8592; Notebook Anterior</button>
        </a>
    </div>
    <div style="text-align: center;">
        <a href="../../solutions/ch_04/solutions.ipynb">
            <button>Soluciones</button>
        </a>
    </div>
    <div style="text-align: right;">
        <a href="../ch_05/1-introduccion_matplotlib.ipynb">
            <button>Capitulo 5 &#8594;</button>
        </a>
    </div>
</div>

<hr>